In [1]:
import pandas as pd
import numpy as np
import urllib.request, urllib.parse
from bs4 import BeautifulSoup

In [2]:
netflix_data = pd.read_csv("../data/Netflix_Titles.csv")
netflix_data.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...


In [3]:
#IMDB Access/Search and Parse HTML to grab title_ID
def grab_ID(title,year,media_type):
    encoding = 'utf-8' #standard internet encoding
    find_url = 'https://www.imdb.com/find' #base search URL for IMDB
    values = {'q':title} # What we are searching for
    query = urllib.parse.urlencode(values).encode(encoding) #parsing the query and encoding correctly for web to understand it
    req = urllib.request.Request(find_url,query) #request: the "link" you can paste in the search bar
    resp = urllib.request.urlopen(req) #sending the request and recording the response
    html = BeautifulSoup(resp,"html") #using Beautiful Soup 4 to process the HTML response
    #Finding top 3 results through <td> tags with result text class in the HTML response. <td> is a table cell in HTML
    search = html.findAll("td", {"class": "result_text"})[:3]
    if len(search) > 0:
        for item in search: 
            if  str(item.text).find(str(year)) > 0:
                if str(media_type) == "TV Show":
                    if str(item.text).find("Series") > 0:
                        title_names = str(item.text)
                        title_IDs = str(item.a["href"])
                        #confidence = 
                        break
                    else:
                        #confidence = 
                        title_names = str(item.text)
                        title_IDs = str(item.a["href"])
                        break
                else:
                    title_names = str(item.text)
                    title_IDs = str(item.a["href"])
                    #confidence =
                    break

            else:
                if str(media_type) == "TV Show":
                    if str(item.text).find("Series") > 0:
                        title_names = str(item.text)
                        title_IDs = str(item.a["href"])
                        #confidence =
                        break
                    else:
                        title_names = str(item.text)
                        title_IDs = str(item.a["href"])
                        #confidence =
                        break
                else:
                    title_names = "Not Found"
                    title_IDs = "Not Found"
                    #confidence =
                    break
    else:
        title_names = "Not Found"
        title_IDs = "Not Found"
        #confidence =

    return title_names, title_IDs

In [5]:
def grab_Rating(title_ID):
    imdb_url = "https://www.imdb.com" #base imdb website
    if title_ID == "Not Found":
        rating = "Not Found"
    else:
        target_url = imdb_url + title_ID #url for title
        target_url = target_url
        resp = urllib.request.urlopen(target_url)
        html = BeautifulSoup(resp,"html") #using Beautiful Soup 4 to process the HTML responce
        search = html.findAll("div", {"class": "ratingValue"})
        if len(search) > 0:
            for item in search: #Grabing rating text
                rating = str(item.strong['title'])
        else:
            rating = "Not Found"

    return rating

In [6]:
# Every run last for 2h. Be aware of that
IMDb_rating = []
IMDb_titleID = []
IMDb_title_name = []
NumShows = len(netflix_data.index)
for i in range(NumShows):
    title_name, title_ID = grab_ID(netflix_data.title[i],netflix_data.release_year[i],netflix_data.type[i])
    IMDB_titleID.append(title_ID)
    IMDB_title_name.append(title_name)
    rating = grab_Rating(title_ID)
    IMDB_rating.append(rating)
    if i%100 == 0:
        print("Progress is: ", i)

Progress is:  0
Progress is:  100
Progress is:  200
Progress is:  300
Progress is:  400
Progress is:  500
Progress is:  600
Progress is:  700
Progress is:  800
Progress is:  900
Progress is:  1000
Progress is:  1100
Progress is:  1200
Progress is:  1300
Progress is:  1400
Progress is:  1500
Progress is:  1600
Progress is:  1700
Progress is:  1800
Progress is:  1900
Progress is:  2000
Progress is:  2100
Progress is:  2200
Progress is:  2300
Progress is:  2400
Progress is:  2500
Progress is:  2600
Progress is:  2700
Progress is:  2800
Progress is:  2900
Progress is:  3000
Progress is:  3100
Progress is:  3200
Progress is:  3300
Progress is:  3400
Progress is:  3500
Progress is:  3600
Progress is:  3700
Progress is:  3800
Progress is:  3900
Progress is:  4000
Progress is:  4100
Progress is:  4200
Progress is:  4300
Progress is:  4400
Progress is:  4500
Progress is:  4600
Progress is:  4700
Progress is:  4800
Progress is:  4900
Progress is:  5000
Progress is:  5100
Progress is:  5200
Progr

In [7]:
output = pd.DataFrame({'IMDb_titleID': IMDb_titleID, 'IMDb_rating': IMDb_rating, 'IMDb_title_name': IMDb_title_name})
output.to_csv("../data/IMDb_results_nov-22-2020.csv")
print("Complete!")

Complete!


In [10]:
output

,IMDB_titleID,IMDB_rating,IMDB_title_name
0,/title/tt9428190/,3.2 based on 294 user ratings,Norm of the North: King Sized Adventure (2019)
1,/title/tt6999080/,5.0 based on 21 user ratings,Jandino: Whatever it Takes (2016) (TV Special)
2,/title/tt1659175/,"7.9 based on 5,331 user ratings","Transformers: Prime (2010) (TV Series) aka ""T..."
3,/title/tt3604232/,6.0 based on 830 user ratings,Transformers: Robots in Disguise (2014) (TV S...
4,/title/tt6119504/,"5.2 based on 5,496 user ratings",#Realityhigh (2017)
...,...,...,...
6229,/title/tt0401747/,"8.3 based on 8,840 user ratings",Red vs. Blue (2003) (TV Series)
6230,/title/tt2520512/,"7.7 based on 4,530 user ratings",Maron (2013) (TV Series)
6231,/title/tt11460334/,6.1 based on 19 user ratings,Little Baby Bum: Nursery Rhyme Friends (2016)...
6232,/title/tt2164430/,"7.9 based on 14,452 user ratings",A Young Doctor's Notebook & Other Stories (20...
